    Conv1D layer -> LSTM

In [14]:
from emotion_recognition import utils
from emotion_recognition import features

import numpy as np

import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras import layers
from keras.models import Sequential
from keras import regularizers
from keras.layers import Bidirectional
from keras.layers import LSTM , RNN

import warnings

warnings.filterwarnings(action='ignore') 

# 1.1 Data preparation
    min_length = 34510
    max_length = 22230
    Setting parameters

In [34]:
window_size = 34000 # 특징 추출을 위한 윈도우 크기
class_size = 4 #class 크기
sr = 16000

# 0 = neutral, 1 = anger, 2 = happiness, 3 = sadness
emotion = ["neutral","anger","happiness","sadness"]

In [17]:
# 파일 이름 읽어오기
path = "../data/"
text_filename = "filelist_wav.txt"

filelist_wav = []
emotionlist = []

f = open(path+text_filename, 'r')

while True:
    line = f.readline()
    if not line: break
    
    filename, label = line.split()
    
    filelist_wav.append(filename)
    emotionlist.append(label)
    
f.close()



In [18]:
Training_filename = []
Training_emotionlist = []
Test_filename = []
Test_emotionlist = []

for i in range(len(filelist_wav)):
    if (filelist_wav[i].split("_")[0][1:] == '1'):
        Test_filename.append(filelist_wav[i])
        Test_emotionlist.append(emotionlist[i])
    elif (filelist_wav[i].split("_")[0][1:] == '6'):
        Test_filename.append(filelist_wav[i])
        Test_emotionlist.append(emotionlist[i])
    elif (filelist_wav[i].split("_")[0][1:] == '7'):
        Test_filename.append(filelist_wav[i])
        Test_emotionlist.append(emotionlist[i])
    elif (filelist_wav[i].split("_")[0][1:] == '11'):
        Test_filename.append(filelist_wav[i])
        Test_emotionlist.append(emotionlist[i])
    else:
        Training_filename.append(filelist_wav[i])
        Training_emotionlist.append(emotionlist[i])

In [19]:
print (np.shape(Training_filename)) # 614개 학습시키고 
print (np.shape(Test_filename))     # 226개 테스트하기 총 840개(파일이름에 1,6,7,11 있는 것)

(614,)
(226,)


In [20]:
Training_vector = []
Training_label = []

for ix in range(len(Training_filename)):
    # 파일 읽어오기
    if Training_emotionlist[ix] == "excitement":
        continue
        
    if Training_emotionlist[ix] == "fear":
        continue
    
    #print (str(ix)+ "\t" + Training_emotionlist[ix] + "\t" + path+'wav/' + Training_filename[ix])
    
    y,sr = utils.loadwav(path+'wav/' + Training_filename[ix])
    
    temp = emotion.index(Training_emotionlist[ix])    
    label = np.zeros(class_size)
    label[temp] = 1
    
    #파일 전체 길이
    length = len(y)
    
    for i in range(0, length-window_size, int(window_size/3)): 
        y_sub = y[i:i+window_size]
            
        Training_vector.append(y_sub) # [[96000개], ........... ,[] ]=>총 204개(전체 파일 자른 개수)
        Training_label.append(label)  # [0,0,0,1]식으로 감정표현
        
print(len(Training_vector[0]))


34000


In [21]:
Test_vector = []
Test_label = []

for ix in range(len(Test_filename)):
    # 파일 읽어오기
    if Test_emotionlist[ix] == "excitement":
        continue
        
    if Test_emotionlist[ix] == "fear":
        continue
    
    print (str(ix)+ "\t" + Test_emotionlist[ix] + "\t" + path+'wav/' + Test_filename[ix])
    
    y,sr = utils.loadwav(path+'wav/' + Test_filename[ix])
    
    temp = emotion.index(Test_emotionlist[ix])    
    label = np.zeros(class_size)
    label[temp] = 1
    
    #파일 전체 길이
    length = len(y)
    
    for i in range(0, length-window_size, int(window_size/3)):
        y_sub = y[i:i+window_size]
            
        Test_vector.append(y_sub)
        Test_label.append(label)

1	sadness	../data/wav/s7_sadness_M_s1.wav
2	happiness	../data/wav/s11_happiness_F_h20.wav
3	sadness	../data/wav/s11_sadness_F_s1.wav
4	sadness	../data/wav/s11_sadness_F_s9.wav
9	neutral	../data/wav/s1_neutral_M_n20.wav
10	sadness	../data/wav/s6_sadness_F_s18.wav
11	anger	../data/wav/s1_anger_M_a4.wav
13	happiness	../data/wav/s6_happiness_F_h14.wav
14	happiness	../data/wav/s11_happiness_F_h15.wav
16	anger	../data/wav/s6_anger_F_a12.wav
18	sadness	../data/wav/s11_sadness_F_s17.wav
19	happiness	../data/wav/s6_happiness_F_h1.wav
21	sadness	../data/wav/s6_sadness_F_s10.wav
24	happiness	../data/wav/s6_happiness_F_h15.wav
26	neutral	../data/wav/s6_neutral_F_n15.wav
27	sadness	../data/wav/s7_sadness_M_s15.wav
28	happiness	../data/wav/s6_happiness_F_h4.wav
29	sadness	../data/wav/s7_sadness_M_s11.wav
30	sadness	../data/wav/s6_sadness_F_s12.wav
31	anger	../data/wav/s7_anger_M_a17.wav
32	neutral	../data/wav/s1_neutral_M_n13.wav
33	anger	../data/wav/s11_anger_F_a16.wav
34	sadness	../data/wav/s7_sad

In [22]:
print (np.shape(Training_vector), np.shape(Test_vector))

(2307, 34000) (1088, 34000)


In [23]:
import math

for i in range(len(Training_vector)):
    Training_vector[i] = Training_vector[i] * 2 / math.pow(2,16)
    
for i in range(len(Test_vector)):
    Test_vector[i] = Test_vector[i] * 2 / math.pow(2,16)

In [24]:
Training_vector = np.expand_dims(Training_vector,axis=2)
Test_vector = np.expand_dims(Test_vector,axis=2)

In [25]:
print (np.shape(Training_vector), np.shape(Test_vector))

(2307, 34000, 1) (1088, 34000, 1)


In [26]:
X_train = np.array(Training_vector)
Y_train = np.array(Training_label)
X_test = np.array(Test_vector)
Y_test = np.array(Test_label)

In [27]:
random_index = np.arange(np.shape(X_train)[0])
np.random.shuffle(random_index)

In [28]:
X_train = X_train[random_index]
Y_train = Y_train[random_index]

In [29]:
print (np.shape(X_train), np.shape(X_test))

(2307, 34000, 1) (1088, 34000, 1)


In [30]:
print (np.shape(Y_train), np.shape(Y_test))

(2307, 4) (1088, 4)


In [36]:
model_m = Sequential()
model_m.add(layers.Conv1D(40, 80, activation='relu', padding='same', input_shape=(window_size, 1))) # Temporal convolution
model_m.add(layers.MaxPooling1D(pool_size=2, strides =2))      # Pooling
model_m.add(layers.Conv1D(40, 4000, padding = 'same',activation='relu')) # Temporal convolution
model_m.add(layers.MaxPooling1D(pool_size=20, strides =20))     # MaxPooling
model_m.add(Bidirectional(LSTM(128, input_shape=(850, 40) ))) #Recurrent layers
model_m.add(layers.Dense(class_size, activation='softmax'))
print(model_m.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_21 (Conv1D)           (None, 34000, 40)         3240      
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 17000, 40)         0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 17000, 40)         6400040   
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 850, 40)           0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 256)               173056    
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 1028      
Total params: 6,577,364
Trainable params: 6,577,364
Non-trainable params: 0
___________________________________________

In [37]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 200
EPOCHS = 400

history = model_m.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      validation_data=(X_test,Y_test),
                      verbose=1)


Train on 2307 samples, validate on 1088 samples
Epoch 1/400


ResourceExhaustedError: OOM when allocating tensor with shape[1,17000,160000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node training/Adam/gradients/conv1d_22/convolution_grad/Conv2DBackpropFilter}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
import matplotlib.pyplot as plt

acc = history.history["acc"]
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs= range(1,len(acc)+1)

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss,'b',label = 'Validation loss')
plt.title('Training an validation loss')
plt.legend()

plt.show()

In [ ]:
acc =model_m.evaluate(X_test,Y_test)

In [ ]:
print (acc[1])